In [ ]:
import os
import preprocessor as prep
import pandas as pd
import unifloc.uniflocpy.uTools.plotly_workflow as pw
import datetime


In [ ]:
well_name = '507'
chess_file_name = 'Скв. 507 (01.07.2018-31.03.2019).xls'

read_initial_data = False
plot_initial_data = False
create_input_data = True
created_input_data_type = 1 # 0 - adaptation input, 1 - restore input with calibation coefficients

In [ ]:
current_path = os.getcwd()
time_mark = datetime.datetime.today().strftime('%Y_%m_%d_%H_%M_%S')
path_to_data = current_path + "\\data\\"
path_to_work_dir = current_path + "\\data\\" + well_name +  "\\"
save_dir_name = 'init_edit'
path_to_save = path_to_work_dir + save_dir_name + '\\'
dirnames_list = []
for (dirpath, dirnames, filenames) in os.walk(path_to_data):
    dirnames_list.extend(dirnames)
    break
print(dirnames_list)

Обработка исходных данных

In [ ]:

if read_initial_data:
    try:
        os.mkdir(path_to_work_dir + save_dir_name)
    except:
        pass
    data_file_path = path_to_work_dir + well_name + ".csv"
    well_data = pd.read_csv(data_file_path,sep=';', header=None)
    well_data = prep.initial_editing(well_data, well_name)
    well_data = prep.create_edited_df(well_data)
    if plot_initial_data:
        plot_file_path = path_to_save + well_name + '_init_sp.html'
        well_data_traces = pw.create_traces_list_for_all_columms(well_data, 'lines+markers', low_memory = True)
        pw.plot_subplots(well_data_traces, plot_file_path, True)
        
    well_data.to_csv(path_to_save + well_name + "_first_edit.csv")

Генерация входных данных в модель для адаптации и восстановления дебитов

In [ ]:
left_boundary = [datetime.datetime(2018,8,3)]#, datetime.datetime(2018,11,29)]
right_boundary = [datetime.datetime(2018,8,5)]#, datetime.datetime(2019,2,28)]

In [ ]:
edited_data_cs = pd.read_csv(path_to_save + well_name + "_first_edit.csv")
edited_data_cs.index = pd.to_datetime(edited_data_cs['Время'])
del edited_data_cs['Время']
edited_data_cs = edited_data_cs.resample('3h').mean()
edited_data_cs['Выходная частота ПЧ'] = edited_data_cs['Выходная частота ПЧ'].fillna(method='ffill')
edited_data_cs['Температура на приеме насоса (пласт. жидкость)']  = \
        edited_data_cs['Температура на приеме насоса (пласт. жидкость)'].fillna(method='ffill')
if created_input_data_type == 0:
    edited_data_cs = edited_data_cs.dropna(subset =['Объемный дебит жидкости'])
edited_data_cs = edited_data_cs.fillna(method='ffill')
edited_data_cs['ГФ'] = edited_data_cs['Объемный дебит газа'] / edited_data_cs['Объемный дебит нефти']
edited_data_cs = prep.mark_df_columns(edited_data_cs, 'СУ')

In [ ]:
chess_data = pd.read_excel(path_to_work_dir + chess_file_name)
chess_data.index = pd.to_datetime(chess_data['Дата'], dayfirst = True, format = "%d.%m.%Y", infer_datetime_format=True)
del chess_data['Дата']
chess_data.index.name = 'Время'
chess_data = chess_data[chess_data.columns[5:]]
chess_data = chess_data.resample('3h').last()
chess_data = chess_data.fillna(method='ffill')
chess_data = prep.mark_df_columns(chess_data, 'Ш')

In [ ]:
if created_input_data_type == 0:
    input_data_dir_name = 'adapt_input_' + time_mark
    path_to_input_data = path_to_work_dir + input_data_dir_name + '\\'
    if create_input_data:
        try:
            os.mkdir(path_to_work_dir + input_data_dir_name)
        except:
            pass
    created_input_data = edited_data_cs.join(chess_data, how = 'outer')
    created_input_data = created_input_data.dropna(subset = ['Объемный дебит жидкости (СУ)'])
    created_input_data = prep.cut_df(created_input_data, left_boundary, right_boundary)
    plot_file_path = path_to_input_data + well_name + '_adapt_input.html'
    input_data_traces = pw.create_traces_list_for_all_columms(created_input_data, 'lines+markers', low_memory = True)
    pw.plot_subplots(input_data_traces, plot_file_path, True)
    
    created_input_data.to_csv(path_to_input_data + well_name + '_adapt_input.csv')

In [ ]:
print(path_to_input_data + well_name + '_adapt_input.csv')
    

In [ ]:
dir_name_with_input_data = 'second_edit' + '\\'
input_data_file_name = '507_adapt_input'
dir_name_with_calculated_data = 'adaptation_2019_11_03_16_43' + '\\'
calculated_data_file_name = '507_adapt_1'

In [ ]:
calculated_data = prep.load_calculated_data_from_csv(path_to_work_dir + dir_name_with_calculated_data +
                                                calculated_data_file_name +  '.csv')

In [ ]:
input_data = pd.read_csv(path_to_work_dir + dir_name_with_input_data + input_data_file_name +  '.csv')
input_data.index = input_data['Время']
del input_data['Время']
all_data = input_data.join(calculated_data, how = 'outer')

all_data.to_csv(path_to_work_dir + dir_name_with_calculated_data + well_name + '_calc_and_input' +  '.csv' )

In [ ]:
calibr_data = calculated_data[['К. калибровки по напору - множитель (Модель)',
                               'К. калибровки по мощности - множитель (Модель)']]
calibr_data = prep.mark_df_columns(calibr_data, 'Подготовленные')
calibr_data = calibr_data.resample('3h').mean()
calibr_data = calibr_data.interpolate()

In [ ]:
if created_input_data_type == 1:
    input_data_dir_name = 'restore_input_' + time_mark
    path_to_input_data = path_to_work_dir + input_data_dir_name + '\\'
    if create_input_data:
        try:
            os.mkdir(path_to_work_dir + input_data_dir_name)
        except:
            pass
    created_input_data = edited_data_cs.join(chess_data, how = 'outer')
    created_input_data = created_input_data.join(calibr_data, how ='inner')
    created_input_data = prep.cut_df(created_input_data, left_boundary, right_boundary)
    plot_file_path = path_to_input_data + well_name + '_restore_input.html'
    input_data_traces = pw.create_traces_list_for_all_columms(created_input_data, 'lines+markers', low_memory = True)
    pw.plot_subplots(input_data_traces, plot_file_path, True)
    created_input_data.to_csv(path_to_input_data + well_name + '_restore_input.csv')